In [9]:
from osgeo import gdal, ogr, osr
import numpy as np
import xarray as xr
import os
import shutil

In [10]:
# 打开nc文件
ds = xr.open_dataset(r'air.2m.gauss.2022.nc') # 打开nc文件
out_netCDF = r'air.2m.gauss.2022_clip.nc' # 输出文件名
shp_path = r'ne_10m_coastline/ne_10m_coastline_plog.shp' # 获取shp文件的路径
lat = ds['lat'].values # 获取经纬度
lon = ds['lon'].values # 获取经纬度
air = ds['air'].values # 获取数据
time = ds['time'].values # 获取时间


In [11]:
# 生成网格信息
# 没有特殊需求的情况下，不需要修改这一段代码
LonMin,LatMax,LonMax,LatMin = [lon.min(),lat.max(),lon.max(),lat.min()]  # 获取经纬度范围
N_Lat = len(lat) # 获取经纬度个数
N_Lon = len(lon) # 获取经纬度个数
Lon_Res = (LonMax - LonMin) /(float(N_Lon)-1) # 获取经纬度分辨率
Lat_Res = (LatMax - LatMin) / (float(N_Lat)-1) # 获取经纬度分辨率

In [12]:
# 创建一个临时文件夹
# 没有特殊需求的情况下，不需要修改这一段代码
'''
我也不知道为什呢，生成多波段的情况下数据会出错，所以我只能生成多个单波段的tif文件
如果有大佬知道怎么解决，欢迎留言
'''
if not os.path.exists('temporary_folders'):
    os.makedirs('temporary_folders')

for i in range(len(air)):
    out_tif = 'temporary_folders/air.2m.gauss.2022_%s.tif' % i # 生成临时tif文件
    tif_ds = gdal.GetDriverByName('Gtiff').Create(out_tif,N_Lon,N_Lat,1,gdal.GDT_Float32) 
    geotransform = (LonMin,Lon_Res, 0, LatMin, 0, Lat_Res) # 设置栅格数据的左上角的经度坐标，经度分辨率，旋转角度，左上角的纬度坐标，纬度分辨率，旋转角度
    tif_ds.SetGeoTransform(geotransform) # 设置栅格数据的地理参考信息
    srs = osr.SpatialReference() # 创建坐标系
    srs.ImportFromEPSG(4326) # 定义地理坐标系统为WGS84(4326代表WGS84)
    tif_ds.SetProjection(srs.ExportToWkt()) # 设置栅格数据的坐标系
    tif_ds.GetRasterBand(1).WriteArray(air[i,:,:]) # 写入数据
    tif_ds.FlushCache() # 将数据写入文件

In [13]:
temporary_folders_files = os.listdir('temporary_folders') # 获取临时文件夹下的文件
temporary_folders_files = [os.path.join('temporary_folders',i) for i in temporary_folders_files] # 获取临时文件夹下的文件路径



if not os.path.exists('temporary_folders_clip'):
    os.makedirs('temporary_folders_clip')

for i in range(len(temporary_folders_files)):
    # ds = gdal.Open(temporary_folders_files[i]) # 打开刚刚输出的tif文件
    # print(temporary_folders_files[i])
    print(r'temporary_folders_clip/air.2m.gauss.2022_clip_%s.tif' % i)
    
    # 使用shp裁剪
    ds = gdal.Warp(
    r'temporary_folders_clip/air.2m.gauss.2022_clip_%s.tif' % i, #待裁剪的影像完整路径（包括文件名）
    temporary_folders_files[i],    #裁剪后图像保存的完整路径（包括文件名）
    format='GTiff', # 保存图像的格式
    cutlineDSName=shp_path, # 矢量文件的完整路径
    cropToCutline=True, # 保证裁剪后影像大小跟矢量文件的图框大小一致（设置为False时，结果图像大小会跟待裁剪影像大小一样，则会出现大量的空值区域）
    # cutlineWhere=cutlineWhere, #矢量文件筛选条件,
    dstNodata=np.nan, #设置空值
    )
ds = None # 关闭文件
    

temporary_folders_clip/air.2m.gauss.2022_clip_0.tif
temporary_folders_clip/air.2m.gauss.2022_clip_1.tif
temporary_folders_clip/air.2m.gauss.2022_clip_2.tif
temporary_folders_clip/air.2m.gauss.2022_clip_3.tif
temporary_folders_clip/air.2m.gauss.2022_clip_4.tif
temporary_folders_clip/air.2m.gauss.2022_clip_5.tif
temporary_folders_clip/air.2m.gauss.2022_clip_6.tif
temporary_folders_clip/air.2m.gauss.2022_clip_7.tif
temporary_folders_clip/air.2m.gauss.2022_clip_8.tif
temporary_folders_clip/air.2m.gauss.2022_clip_9.tif
temporary_folders_clip/air.2m.gauss.2022_clip_10.tif
temporary_folders_clip/air.2m.gauss.2022_clip_11.tif
temporary_folders_clip/air.2m.gauss.2022_clip_12.tif
temporary_folders_clip/air.2m.gauss.2022_clip_13.tif
temporary_folders_clip/air.2m.gauss.2022_clip_14.tif
temporary_folders_clip/air.2m.gauss.2022_clip_15.tif
temporary_folders_clip/air.2m.gauss.2022_clip_16.tif
temporary_folders_clip/air.2m.gauss.2022_clip_17.tif
temporary_folders_clip/air.2m.gauss.2022_clip_18.tif
tem

In [14]:
# 重新生成NC文件

file_list = os.listdir('temporary_folders_clip') # 获取裁剪后的tif文件
file_list = [os.path.join('temporary_folders_clip',i) for i in file_list] # 获取裁剪后的tif文件路径
file_list

ds0 = gdal.Open(file_list[0]) # 打开第一个tif文件
data = ds0.ReadAsArray() # 读取数据
geotransform = ds0.GetGeoTransform() # 获取栅格数据的地理参考信息
num_x = ds0.RasterXSize # 获取栅格数据的列数
num_y = ds0.RasterYSize # 获取栅格数据的行数

print(num_x,num_y,'\n',geotransform)


32 27 
 (75.0, 1.875, 0.0, 54.267677307128906, 0.0, -1.9041290283203125)


In [15]:
# 生成网格信息
# 没有特殊需求的情况下，不需要修改这一段代码

lon = np.arange(geotransform[0],geotransform[0]+num_x*geotransform[1],geotransform[1]) # 生成经度

lat = np.arange(geotransform[3],geotransform[3]+num_y*geotransform[5],geotransform[5]) # 生成纬度

air_clip_nc = np.zeros((len(file_list),num_y,num_x)) # 生成一个空数组

# 获取数据
ds0_data = ds0.ReadAsArray() # 读取数据

# 获取array的序列
index_ = int(file_list[0].split('_')[-1].split('.')[0])
air_clip_nc[index_,:,:] = ds0_data

In [16]:
for i in range(1,len(file_list)):
    ds = gdal.Open(file_list[i]) # 打开tif文件
    ds_data = ds.ReadAsArray() # 读取数据
    index_ = int(file_list[i].split('_')[-1].split('.')[0])
    air_clip_nc[index_,:,:] = ds_data
    ds = None # 关闭文件

# 生成nc文件
result_nc = xr.Dataset()
result_nc['air'] = xr.DataArray(air_clip_nc,coords=[('time',time),('lat',lat),('lon',lon)],dims=['time','lat','lon'])
result_nc['air'].attrs['long_name'] = 'air'
result_nc['air'].attrs['units'] = 'K'
result_nc['air'].attrs['standard_name'] = '2m_air_temperature'
result_nc['lon'].attrs['long_name'] = 'longitude'
result_nc['lon'].attrs['units'] = 'degrees_east'
result_nc['lon'].attrs['standard_name'] = 'longitude'
result_nc['lat'].attrs['long_name'] = 'latitude'
result_nc['lat'].attrs['units'] = 'degrees_north'
result_nc['lat'].attrs['standard_name'] = 'latitude'
result_nc['time'].attrs['long_name'] = 'time'

result_nc.to_netcdf(out_netCDF) # 保存nc文件

# 删除临时文件夹
try:
    shutil.rmtree('temporary_folders_clip')
    shutil.rmtree('temporary_folders')
except PermissionError:
    print('请关闭Python后手动删除临时文件夹')
    # 通常情况下，temporary_folders文件夹最后一个文件会被占用，无法删除，此时需要手动删除
    # 但是不影响裁剪程序的运行
    # 关闭python之后手动删除一下啦
    pass

请关闭Python后手动删除临时文件夹
